In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from transformers import MBart50TokenizerFast,MBartForConditionalGeneration,Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset, load_metric
import numpy as np
import evaluate

In [3]:
raw_datasets = load_dataset("Helsinki-NLP/opus-100", "en-id")
model_mbart = 'facebook/mbart-large-50-one-to-many-mmt'

In [4]:
tokenizer = MBart50TokenizerFast.from_pretrained(model_mbart,src_lang="en_XX",tgt_lang = "id_ID")

In [5]:
source_lang = "en"
target_lang = "id"

def preprocess(data):
  inputs = [dt[source_lang] for dt in data["translation"]]
  targets = [dt[target_lang] for dt in data["translation"]]
  model_inputs = tokenizer(inputs, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

tokenized_datasets = raw_datasets.map(preprocess, batched=True)

In [6]:
# Shuffle the train dataset and select 25% of it
small_train_dataset = (
    tokenized_datasets["train"]
    .shuffle(seed=42)
    .select(range(len(tokenized_datasets["train"]) // 10))
)

# Shuffle the test dataset and select 25% of it
small_eval_dataset = (
    tokenized_datasets["test"]
    .shuffle(seed=42)
    .select(range(len(tokenized_datasets["test"]) // 10))
)

In [7]:
len(small_train_dataset)

100000

In [8]:
model = MBartForConditionalGeneration.from_pretrained(model_mbart)
model = model.cuda()

In [9]:
hyperparameters = {
    'learning_rate': 1e-5,
    'batch_size': 8,
    'num_epochs': 5
}

args = Seq2SeqTrainingArguments(
    f"mbart-large-50-one-to-many-mmt-finetuned-en-to-id",
    evaluation_strategy="epoch",
    learning_rate=hyperparameters['learning_rate'],
    per_device_train_batch_size=hyperparameters['batch_size'],
    per_device_eval_batch_size=hyperparameters['batch_size'],
    weight_decay=0.01,
    save_total_limit=hyperparameters['num_epochs'],
    num_train_epochs=hyperparameters['num_epochs'],
    predict_with_generate=True,
)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
metric = evaluate.load("sacrebleu")
meteor = evaluate.load('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(
        decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels)
    meteor_result = meteor.compute(
        predictions=decoded_preds, references=decoded_labels)
    prediction_lens = [np.count_nonzero(
        pred != tokenizer.pad_token_id) for pred in preds]
    result = {'bleu': result['score']}
    result["gen_len"] = np.mean(prediction_lens)
    result["meteor"] = meteor_result["meteor"]
    result = {k: round(v, 4) for k, v in result.items()}
    return result

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

  0%|          | 0/62500 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


{'loss': 2.0911, 'grad_norm': 5.785200595855713, 'learning_rate': 9.920000000000002e-06, 'epoch': 0.04}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


{'loss': 1.8662, 'grad_norm': 6.672329902648926, 'learning_rate': 9.84e-06, 'epoch': 0.08}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


{'loss': 1.0378, 'grad_norm': 7.40623664855957, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 1.4611365795135498, 'eval_bleu': 30.8192, 'eval_gen_len': 12.38, 'eval_meteor': 0.5431, 'eval_runtime': 11.4011, 'eval_samples_per_second': 17.542, 'eval_steps_per_second': 2.193, 'epoch': 5.0}
{'train_runtime': 20884.4541, 'train_samples_per_second': 23.941, 'train_steps_per_second': 2.993, 'train_loss': 1.3045448422851562, 'epoch': 5.0}


TrainOutput(global_step=62500, training_loss=1.3045448422851562, metrics={'train_runtime': 20884.4541, 'train_samples_per_second': 23.941, 'train_steps_per_second': 2.993, 'total_flos': 2.476985304927437e+16, 'train_loss': 1.3045448422851562, 'epoch': 5.0})

In [13]:
trainer.save_model('opus-mt-en-id-finetuned-en-to-id')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


In [14]:
src_text = [
    "Kelas NLP adalah kelas yang sangat menyenangkan, saya senang belajar NLP. Pak Hendrik merupakan dosen terbaik di program studi IBDA 2020 Calvin Institut of Technology."
]
model_path = "opus-mt-en-id-finetuned-en-to-id"

tokenizer = MBart50TokenizerFast.from_pretrained(model_path, src_lang="en_XX")
model = MBartForConditionalGeneration.from_pretrained(model_path)

model_inputs = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["id_ID"],
    max_new_tokens=360
)
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(src_text)
print(translation)

['Kelas NLP adalah kelas yang sangat menyenangkan, saya senang belajar NLP. Pak Hendrik merupakan dosen terbaik di program studi IBDA 2020 Calvin Institut of Technology.']
['Kelas NLP adalah kelas yang sangat menyenangkan, saya senang belajar NLP. Pak Hendrik merupakan dosen terbaik di program studi IBDA 2020 Calvin Institute of Technology.']
